In [11]:
import folium
from folium.plugins import TimestampedGeoJson
from folium import IFrame
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import numpy as np


In [12]:
ds_path = r'C:\Users\User\VisualStudioProjects\controllingDKWpriv\controllingDKWpriv\NewReports\md2.xlsx'
df_stores = pd.read_excel(ds_path, sheet_name='ds')
df_stores['Longitude'] = df_stores['Longitude'].str.replace(',','.')
df_stores['Latitude'] = df_stores['Latitude'].str.replace(',','.')
df_stores['Longitude'] = pd.to_numeric(df_stores['Longitude'], errors='coerce')
df_stores['Latitude'] = pd.to_numeric(df_stores['Latitude'], errors='coerce')

df_stores['Store Opening Date'] = pd.to_datetime( df_stores['Store Opening Date'], format='%d.%m.%Y')

df_stores.head()

,Site,Site.1,Longitude,Latitude,Postal Code,Store Opening Date,Type of Concept,Site / PC Status,Counter,Makroregion,Region
0,P201,"Gdańsk, Matarnia",18.51856,54.37219,80-298,2016-04-15,SC30,Comp.,1.0,Region 5,North 1
1,P202,"Gdańsk, City Meble",18.58977,54.38919,80-266,2000-05-27,SC30,Comp.,1.0,Region 5,North 6
2,P203,"Poznań, Franowo",16.98776,52.38063,61-285,2013-09-25,SC30,Comp.,1.0,Region 2,Central 1
3,P204,"Szczecin, Wiosenna",14.66207,53.38535,70-807,2009-05-08,SC30,Comp.,1.0,Region 5,North 4
4,P205,"Szczecin, Mieszka",14.52453,53.41297,71-011,2001-03-03,SC30,Comp.,1.0,Region 5,North 4


In [13]:
mask = df_stores[['Site', 'Longitude', 'Latitude', 'Store Opening Date']].notna().all(axis=1)
df = df_stores[mask]
df = df[['Site', 'Longitude', 'Latitude', 'Store Opening Date']]
df = df.sort_values('Store Opening Date', ascending=True)


In [14]:

df= pd.read_excel(r'C:\Users\User\VisualStudioProjects\controllingDKWpriv\controllingDKWpriv\NewReports\dfv.xlsx')
df.head()

,Unnamed: 0,Site,Longitude,Latitude,Store Opening Date,value
0,38,P240,23.13275,53.13862,2003-09-30,10.980110
1,41,P243,20.90645,52.23998,2003-10-25,21.546497
2,42,P244,16.83961,52.35381,2003-12-13,27.150112
3,44,P246,15.12785,51.63961,2004-09-30,30.104564
4,45,P247,19.35008,51.66521,2004-11-19,33.058719


DZIAŁAJĄCA WERSJA

In [15]:
import pandas as pd
import folium

# Tworzenie mapy
m = folium.Map(location=[52.1000, 20.8000], zoom_start=6)


# Dodawanie TimestampedGeoJson do mapy dla animacji
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['Longitude'], row['Latitude']],
        },
        'properties': {
            'time': row['Store Opening Date'].isoformat(),
            'popup': row['Site'],
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'blue',
                'fillOpacity': 0.8,
                'stroke': 'false',
                'radius': 7
            }
        }
    } for idx, row in df.iterrows()
]

# Użycie TimestampedGeoJson
TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    },
    period='P1M',
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=50,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(m)

# Zapisywanie mapy z animacją do nowego pliku HTML
m.save('animated_map_with_dates.html')


MODYFIKACJE PONIŻEJ

1. DZIAŁAJĄCA MAPA Z OKIENKIEM ZE ZMIENIAJĄCĄ SIĘ DATĄ

In [16]:
import pandas as pd
import folium

# Tworzenie mapy
m = folium.Map(location=[52.1000, 20.8000], zoom_start=6)

def create_legend():
    legend_html = '''
    <div style="position: fixed; 
         bottom: 200px; left: 20px; width: 180px; height: auto; 
         border:2px solid grey; z-index:9999; font-size:14px; 
         background-color: white; padding: 5px;">
         &nbsp; Legend: <br>
         <div id="date-display">Data: N/A</div>
    </div>'''
    return legend_html

script = '''
<script type="text/javascript">
    var dateDisplay = document.getElementById('date-display');
    function updateLegendDate() {
        // Użyj ogólnego selektora do znalezienia elementu daty, zamiast polegać na konkretnej ścieżce
        var slider = document.querySelector('a.leaflet-control-timecontrol.timecontrol-date');
        var currentDate = slider ? slider.innerHTML : 'No date';
        dateDisplay.innerHTML = 'Data: ' + currentDate;
    }
    // Cykliczne sprawdzanie i aktualizacja daty co 1 sekundę
    setInterval(updateLegendDate, 100);
</script>
'''

# Dodawanie TimestampedGeoJson do mapy dla animacji
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['Longitude'], row['Latitude']],
        },
        'properties': {
            'time': row['Store Opening Date'].isoformat(),
            'popup': row['Site'],
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'blue',
                'fillOpacity': 0.8,
                'stroke': 'false',
                'radius': 7
            }
        }
    } for idx, row in df.iterrows()
]

# Użycie TimestampedGeoJson
TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    },
    period='P1M',
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=50,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(m)

# Przykładowe użycie funkcji
legend = create_legend()
m.get_root().html.add_child(folium.Element(legend))
# Dodanie skryptu do mapy
m.get_root().html.add_child(folium.Element(script))
# Zapisywanie mapy z animacją do nowego pliku HTML
m.save('animated_map_with_dates_with_window.html')

2 DZIAŁAJĄCA MAPA Z OKIENKIEM ZE ZMIENIAJĄCĄ SIĘ DATĄ I WYKRESEM

In [1]:
import pandas as pd
import numpy as np
import folium
import json

df_chart = pd.DataFrame()
# DataFrame z datami i wartościami
df_chart[['Date', 'Value']] = df[['Store Opening Date', 'value']]
df_chart['Date'] = pd.to_datetime( df_chart['Date'], format='%Y-%m-%d')

df_chart['Date'] = df_chart['Date'].dt.strftime('%Y-%m-%d')
# Konwersja DataFrame do formatu JSON
chart_data_json = df_chart.to_json(orient='records')

# Tworzenie mapy
m = folium.Map(location=[52.1000, 20.8000], zoom_start=6.2)

def create_legend():
    legend_html = '''
    <div style="position: fixed; 
         top: 50; left: 600px; width: 200; height: 50; 
         border:2px solid grey; z-index:9999; font-size:14px; 
         background-color: white; padding: 5px;">
         <div id="date-display">Data: N/A</div>
    </div>'''
    return legend_html

def create_chart_window():
    chart_html = '''
    <div style="position: fixed; top: 50px; left: 20px; width: 400px; height: 200px; border:2px solid grey; z-index:9999; font-size:14px; background-color: white; padding: 5px;">
         <div id="chart-container" style="width: 380px; height: 180px;"></div>
    </div>'''
    return chart_html
plotly_script = '<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>'

# JavaScript do aktualizacji wykresu

script = f'''
<script type="text/javascript">
    var chartData = {chart_data_json};
    var accumulatedData = [];
    
    function updateChart(currentDate) {{
        var newFilteredData = chartData.filter(d => new Date(d.Date) <= new Date(currentDate));
        accumulatedData = newFilteredData;
        var trace1 = {{
            x: accumulatedData.map(d => d.Date),
            y: accumulatedData.map(d => d.Value),
            mode: 'lines', // Usunięcie punktów, pokazanie tylko linii
            line: {{
                color: '#007BFF', // Dodanie koloru linii, np. niebieskiego
                width: 3 // Grubość linii
            }}
        }};

        var annotations = [];
        if (accumulatedData.length > 0) {{
            var lastDataPoint = accumulatedData[accumulatedData.length - 1];
            annotations.push({{
                x: lastDataPoint.Date,
                y: lastDataPoint.Value,
                xref: 'x',
                yref: 'y',
                text: lastDataPoint.Value.toFixed(1),
                showarrow: false,
                arrowhead: 7,
                ax: 0,
                ay: -40,
                font: {{
                    family: 'Arial',
                    size: 16,
                    color: 'black',
                    weight: 'bold'
                }},
        bgcolor: 'white', // Tło adnotacji
        bordercolor: 'black', // Kolor obramowania adnotacji
        borderpad: 5, // Padding obramowania
        borderRadius: 5, // Zaokrąglenie narożników adnotacji
            }});
        }}

        var layout = {{
            title: 'Turnover',
            xaxis: {{
                title: 'Year',
                tickformat: '%Y', // Formatowanie etykiety osi X do pokazywania tylko lat
                showgrid: false, // Usunięcie siatki osi X
                type: 'date'
            }},
            yaxis: {{
                title: 'MPLN',
                showgrid: false // Usunięcie siatki osi Y
            }},
            autosize: true, // Dostosowanie rozmiaru wykresu do kontenera
            margin: {{ l: 50, r: 30, t: 30, b: 50 }}, // Marginesy
            paper_bgcolor: 'white', // Kolor tła wykresu
            plot_bgcolor: 'white', // Kolor tła obszaru danych
            annotations: annotations
        }};
        Plotly.newPlot('chart-container', [trace1], layout);
    }};

    function updateLegendDate() {{
        var slider = document.querySelector('a.leaflet-control-timecontrol.timecontrol-date');
        var currentDate = slider ? slider.innerHTML : 'No date';
        document.getElementById('date-display').innerHTML = '<span style="font-weight: bold; color: black; font-size: 24px;">' + currentDate + '</span>';
        updateChart(currentDate);
    }}
    setInterval(updateLegendDate, 100);
</script>

'''

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['Longitude'], row['Latitude']],
        },
        'properties': {
            'time': row['Store Opening Date'].isoformat(),
            'popup': row['Site'],
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'blue',
                'fillOpacity': 0.8,
                'stroke': 'false',
                'radius': 7
            }
        }
    } for idx, row in df.iterrows()
]

# Użycie TimestampedGeoJson
TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    },
    period='P1M',
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=50,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(m)

# Dodanie elementów HTML i skryptów do mapy

m.get_root().header.add_child(folium.Element(plotly_script))
m.get_root().html.add_child(folium.Element(script))
m.get_root().html.add_child(folium.Element(create_legend()))
m.get_root().html.add_child(folium.Element(create_chart_window()))

# Zapisywanie mapy z animacją do nowego pliku HTML
m.save('animated_map_with_dates_with_window_and_chart.html')


NameError: name 'df' is not defined

In [29]:
chart_data_json

'[{"Date":"2003-09-30","Value":10.9801098928},{"Date":"2003-10-25","Value":21.5464971123},{"Date":"2003-12-13","Value":27.1501116538},{"Date":"2004-09-30","Value":30.1045638897},{"Date":"2004-11-19","Value":33.0587190513},{"Date":"2004-12-03","Value":35.5017426511},{"Date":"2004-12-04","Value":36.8390388092},{"Date":"2005-03-15","Value":38.3670390104},{"Date":"2005-05-06","Value":39.4523587237},{"Date":"2005-05-07","Value":40.3743857563},{"Date":"2005-06-18","Value":41.437616522},{"Date":"2005-07-01","Value":43.2189617902},{"Date":"2005-08-27","Value":43.8484081625},{"Date":"2005-08-27","Value":44.3142559576},{"Date":"2005-08-31","Value":45.1725208498},{"Date":"2005-10-22","Value":45.7091937156},{"Date":"2005-11-23","Value":46.4768512455},{"Date":"2005-11-24","Value":46.7159276822},{"Date":"2005-12-03","Value":47.6476054002},{"Date":"2006-03-04","Value":48.4432362194},{"Date":"2006-03-11","Value":48.9492793473},{"Date":"2006-07-29","Value":49.5418495862},{"Date":"2006-08-05","Value":49

In [33]:
dfj = pd.read_json(chart_data_json)
dfj.head(3)

C:\Users\User\AppData\Local\Temp\ipykernel_1340\828119384.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfj = pd.read_json(chart_data_json)


,Date,Value
0,2003-09-30,10.980110
1,2003-10-25,21.546497
2,2003-12-13,27.150112


In [34]:
df.head(3)

,Unnamed: 0,Site,Longitude,Latitude,Store Opening Date,value
0,38,P240,23.13275,53.13862,2003-09-30,10.980110
1,41,P243,20.90645,52.23998,2003-10-25,21.546497
2,42,P244,16.83961,52.35381,2003-12-13,27.150112


In [35]:
merg = dfj.merge(df, how='left', left_on='Date', right_on='Store Opening Date')
merg

,Date,Value,Unnamed: 0,Site,Longitude,Latitude,Store Opening Date,value
0,2003-09-30,10.980110,38,P240,23.13275,53.13862,2003-09-30,10.980110
1,2003-10-25,21.546497,41,P243,20.90645,52.23998,2003-10-25,21.546497
2,2003-12-13,27.150112,42,P244,16.83961,52.35381,2003-12-13,27.150112
3,2004-09-30,30.104564,44,P246,15.12785,51.63961,2004-09-30,30.104564
4,2004-11-19,33.058719,45,P247,19.35008,51.66521,2004-11-19,33.058719
...,...,...,...,...,...,...,...,...
327,2024-01-03,75.961492,295,P502,14.88598,53.14755,2024-01-03,75.961492
328,2024-01-10,75.743264,296,P503,17.70335,52.84987,2024-01-10,75.743264
329,2024-03-28,75.843623,298,P505,17.71277,51.29909,2024-03-28,75.843623
330,2024-03-29,76.317039,299,P506,16.21605,54.16539,2024-03-29,76.317039


In [ ]:
df_chart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Store Opening Date  278 non-null    datetime64[ns]
 1   value               278 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.5 KB


In [3]:
def decorator(func):
    def wrapper():
        print("Before function execution")
        func()
        print("After function execution")
    return wrapper
@decorator
def my_function():
    print("Inside the function")
my_function()

Before function execution
Inside the function
After function execution
